In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import os

import time
# Khởi tạo trình duyệt
driver = webdriver.Chrome()

# Mở trang web cần tương tác
driver.get('https://www.evn.com.vn/c3/thong-tin-ho-thuy-dien/Muc-nuoc-cac-ho-thuy-dien-117-123.aspx')

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe[src='https://phongchongthientai.evn.com.vn/PageHoChuaThuyDienEmbedEVN.aspx']"))
)

iframe = driver.find_element(By.CSS_SELECTOR, "iframe[src='https://phongchongthientai.evn.com.vn/PageHoChuaThuyDienEmbedEVN.aspx']")
driver.switch_to.frame(iframe)

def chon_vung(ten_v):
    vung = driver.find_element(By.XPATH, '//*[@id="form"]/div[3]/div/div[2]/div/section[1]/div/button')
    vung.click()
    checkbox = driver.find_elements(By.CLASS_NAME, 'checkbox')
    for check in checkbox:
        if check.text == ten_v:
            check.click()

        
def lay_du_lieu(df):
    tentinh = driver.find_elements(By.CSS_SELECTOR, 'b')
    tinh = [element.text for element in tentinh]
    tinh = np.array(tinh)
    tinh = tinh[tinh != '']
    tinh = tinh.reshape(-1, 1)
    
    thongtin = driver.find_elements(By.CLASS_NAME, 'tdclass')
    tt = [element.text for element in thongtin]
    tt = np.array(tt)
    tt = np.reshape(tt, (len(tt)//10, 10))

    dulieu = np.concatenate((tinh, tt), axis=1)
    df_temp = pd.DataFrame(dulieu, columns= name_columns)
    df = pd.concat([df, df_temp], ignore_index=True)
    return df
    

def lay_tu_ngay(day, month, year, df):
    start_date = datetime(year, month, day)
    end_date = datetime(2024, 7, 16)
    current_date = start_date
    while current_date <= end_date:
        date = driver.find_element(By.ID, "UCViewHoChuaThuyDienPublic1_tbxDenNgay")
        date.clear()
        date.send_keys(current_date.strftime("%d/%m/%Y"), ' 23:00')
        button = driver.find_element(By.XPATH, '//*[@id="form"]/div[3]/div/div[2]/div/section[5]/a')
        button.click()
        current_date += timedelta(days=1)
        time.sleep(2)
        df = lay_du_lieu(df)
    return df




name_columns = ['tên hồ', 'Thời điểm', 'htl', 'dbt', 'hc', 'Qve', 'Tqc', 'Qxt', 'Qxm', 'Ncxs', 'Ncxm']

#Nơi lưu file csv
file_path = 'F:/congcu/ML/tay_bac_bo.csv'



if os.path.exists(file_path):
    data = pd.read_csv(file_path)
else:
    data = pd.DataFrame(columns=name_columns)
    
#Chọn vùng để lấy dữ liệu
chon_vung('Tây Bắc Bộ')
#Chọn ngày bắt đầu lấy       
data = lay_tu_ngay(1, 1, 2024, data)

driver.switch_to.default_content()
# Đóng trình duyệt
driver.quit()
data.to_csv(file_path, index=False)